<a href="https://colab.research.google.com/github/vitaliy-sharandin/data_science_projects/blob/master/portfolio/regression/Energy_price_demand_and_potential_supply.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Energy demand prediction of Spain's major cities

This project uses energy demand information from major Spanish cities to predict cumulative energy consumption.

# Dataset
Spanish major cities energy consumption and weather dataset
* https://www.kaggle.com/datasets/nicholasjhana/energy-consumption-generation-prices-and-weather



In [ ]:
!pip install -U -q datasets
!pip install -U -q ydata-profiling
!pip install -U -q feature_engine
!pip install -U -q Boruta
!pip install -U -q optuna
!pip install -U -q eli5
!pip install -U -q statsforecast
!pip install -U -q mlforecast
!pip install -U -q neuralforecast
!pip install -U -q datasetsforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.5 MB/s eta 0:00:00
ERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.5/357.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.5/679.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 16.9 MB/s eta 0:00:00
ERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 5.3 MB/s eta 0:00:00
ERROR: Operation cancelled by user
^C
ERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 21.5 MB/s eta 0:00:0

# EDA


In [ ]:
from datasets import load_dataset
from ydata_profiling import ProfileReport
import pandas as pd
from feature_engine.encoding import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from boruta import BorutaPy
import xgboost as xgb
import optuna
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from datasetsforecast.losses import rmse
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA
from mlforecast import MLForecast
from sklearn.preprocessing import MinMaxScaler
from neuralforecast import NeuralForecast
from neuralforecast.models import LSTM
from neuralforecast.losses.pytorch import DistributionLoss

In [ ]:
energy_consumption_dataset = load_dataset("vitaliy-sharandin/energy-consumption-hourly-spain")
energy_consumption_weather_dataset = load_dataset("vitaliy-sharandin/energy-consumption-weather-hourly-spain")
energy_df = energy_consumption_dataset['train'].to_pandas()
weather_df = energy_consumption_weather_dataset['train'].to_pandas()

## Energy dataset

In [ ]:
energy_df['time'] = pd.to_datetime(energy_df['time'], errors="coerce", utc=True)
energy_df = energy_df.set_index('time')
energy_df.index = pd.to_datetime(energy_df.index,utc=True)
energy_df=energy_df.asfreq('h')

In [ ]:
# profile = ProfileReport(energy_df, title="Energy data report", dark_mode=True)
# profile.to_notebook_iframe()

In [ ]:
def categorize_variables(target, df_train, cat_numeric_unique_threshold=10):
  target = target
  categorical_numeric = [var for var in df_train.columns if df_train[var].dtype!='O' and var!=target and df_train[var].nunique()<=cat_numeric_unique_threshold]
  continuous = [var for var in df_train.columns if df_train[var].dtype!='O' and var!=target and var not in categorical_numeric]
  mixed = [var for var in df_train.columns if pd.api.types.infer_dtype(df_train[var]) == 'mixed']
  categorical_object = [var for var in df_train.columns if df_train[var].dtype=='O' and var not in mixed]
  sorted_features = [target]+categorical_numeric+continuous+categorical_object+mixed
  print('Total columns: '+str(df_train.columns.size)+'\nColumns after sorting: '+str(len(sorted_features)))
  return target, categorical_numeric, continuous, mixed, categorical_object
target, categorical_numeric, continuous, mixed, categorical_object = categorize_variables('total load actual', energy_df)

In [ ]:
energy_df[target] = energy_df[target].interpolate(method='linear')
energy_df = energy_df[target].to_frame()

**Insights**
* We left target variable as the only one, because we want to predict solely based on weather.
* Target variable had several missing values, so we used linear interpolation to fill them.

## Target variable analysis

In [ ]:
target_decompose = seasonal_decompose(energy_df[:168][target], model='additive')
plot = target_decompose.plot()
plot.set_size_inches((16, 9))
plt.show()

In [ ]:
plot_acf(energy_df[target], lags=5)
plt.show()
plot_pacf(energy_df[target], lags=5)
plt.show()

**Insights**
* Series are non-stationary.
* Target variable has daily seasonality.
* The dataset has properties of AR process what stems from fast cutoff in PACF chart, this will become handy when we test ARIMA based models.

## Weather dataset EDA

In [ ]:
weather_df['dt_iso'] = pd.to_datetime(weather_df['dt_iso'], utc=True)

In [ ]:
# profile = ProfileReport(weather_df, title="Weather data report")
# profile.to_notebook_iframe()

In [ ]:
weather_df.drop(['rain_3h','weather_main','weather_icon'], axis=1, inplace=True)
weather_df = weather_df[weather_df['dt_iso'].isin(energy_df.index) & (~weather_df.duplicated(['dt_iso','city_name']))]

In [ ]:
weather_df = weather_df.set_index(['dt_iso','city_name'])
weather_df = weather_df.unstack('city_name')
weather_df.columns = ['_'.join(col).strip() for col in weather_df.columns.values]

#### Merging energy and weather dataset

In [ ]:
energy_weather_df = energy_df.join(weather_df, how='inner')

# Feature selection and engineering

## Transforming variables

In [ ]:
target, categorical_numeric, continuous, mixed, categorical_object = categorize_variables('total load actual', energy_weather_df)

train = energy_weather_df[:-168]
test = energy_weather_df[-168:]

encoder = OrdinalEncoder(
    variables=categorical_object,
    encoding_method='ordered'
)

train = encoder.fit_transform(train, train[target])
test = encoder.fit_transform(test, test[target])

## Feature selection

In [ ]:
from xgboost import XGBRegressor
import eli5

# params = {
#     'objective': 'reg:squarederror',
#     'random_state': 42,
#     'n_jobs': -1,
#     'learning_rate': 0.1,
#     'max_depth': 3,
#     'min_child_weight': 1,
#     'gamma': 0,
#     'subsample': 0.8,
#     'colsample_bytree': 0.8,
#     'reg_alpha': 0,
#     'reg_lambda': 1
# }

xgb = XGBRegressor()
xgb.fit(train[train.columns.difference([target])], train[target])

display(eli5.show_weights(xgb, feature_names = test[test.columns.difference([target])].columns.tolist()))

In [ ]:
# %%time
# lgb_regressor = lgb.LGBMRegressor()

# feat_selector = BorutaPy(lgb_regressor, n_estimators='auto', random_state=42)
# feat_selector.fit(X_train.values, y_train.values)
# selected_rf_features = pd.DataFrame({'Feature':list(X_train.columns),
#                                        'Ranking':feat_selector.ranking_}).sort_values(by='Ranking')
# selected_rf_features.nsmallest(40, 'Ranking').plot.barh(x='Feature',figsize=(24,5))

# Model selection

First, we transform data into statsforecast format and also order columns for statsforecast framework so that it can use

In [ ]:
def transform_to_statsforecast_format(df):
  energy_weather_df_forecast = df.copy()
  energy_weather_df_forecast['unique_id'] = 'Energy_weather_Spain'
  energy_weather_df_forecast['ds'] = energy_weather_df_forecast.index.tz_localize(None)
  energy_weather_df_forecast = energy_weather_df_forecast.rename(columns={'total load actual':'y'})

  exogenous_variables = energy_weather_df_forecast.columns.difference(['unique_id', 'ds', 'y']).tolist()
  cols = ['unique_id', 'ds', 'y'] + exogenous_variables
  return energy_weather_df_forecast.reindex(columns=cols)

train_forecast = transform_to_statsforecast_format(train)[-168:]
test_forecast = transform_to_statsforecast_format(test)

## ARIMA and others with statsforecast model test

Let's see how ARIMA with exogenous variables performs.

In [ ]:
# season_length = 24

# models = [
#     AutoARIMA(season_length=season_length)
# ]

# sf = StatsForecast(
#     models=models,
#     freq='H',
#     n_jobs=-1, verbose=True
# )

# crossvalidation_df = sf.cross_validation(
#     df = train_forecast,
#     h = len(test_forecast.index),
#     n_windows = 3
#   )

# rmse = crossvalidation_df.groupby(['unique_id', 'cutoff']).apply(lambda df: rmse(df['y'], df['AutoARIMA'])).mean()

# display("RMSE: "+str(rmse))

In [ ]:
# cutoff = crossvalidation_df['cutoff'].unique()

# for k in range(len(cutoff)):
#     cv = crossvalidation_df[crossvalidation_df['cutoff'] == cutoff[k]]
#     StatsForecast.plot(crossvalidation_df, cv.loc[:, cv.columns != 'cutoff'])

Now let's see how ARIMA performs without exogenous variables just using target values for training.

In [ ]:
# crossvalidation_df = sf.cross_validation(
#     df = train_forecast[['unique_id', 'ds', 'y']],
#     h = len(test_forecast.index),
#     n_windows = 3
#   )

In [ ]:
# cutoff = crossvalidation_df['cutoff'].unique()

# for k in range(len(cutoff)):
#     cv = crossvalidation_df[crossvalidation_df['cutoff'] == cutoff[k]]
#     StatsForecast.plot(crossvalidation_df, cv.loc[:, cv.columns != 'cutoff'])

As a result we can see that exogenous variables helped us to get a better RMSE result. So, it makes sense to use them.

## Tree model tests with mlforecast framework

Tree model hyperparameters search

In [ ]:
# %%time
# warnings.filterwarnings('ignore')

# def objective(trial):

#     model_name = trial.suggest_categorical("classifier", ['LGBMRegressor', 'XGBRegressor'])

#     if model_name == "LGBMRegressor":
#       params = {
#         "objective": "regression",
#         "metric": "rmse",
#         "n_estimators": 1000,
#         "verbosity": -1,
#         "bagging_freq": 1,
#         "learning_rate": trial.suggest_float("learning_rate_light", 1e-3, 0.1, log=True),
#         "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
#         "subsample": trial.suggest_float("subsample_light", 0.05, 1.0),
#         "colsample_bytree": trial.suggest_float("colsample_bytree_light", 0.05, 1.0),
#         "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
#       }
#       model = lgb.LGBMRegressor(**params)

#     elif model_name == "XGBRegressor":
#       params = {
#       'max_depth': trial.suggest_int('max_depth', 1, 10),
#       'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
#       'n_estimators': trial.suggest_int('n_estimators', 50, 500),
#       'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
#       'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
#       'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
#       'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
#       'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
#       'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
#       'eval_metric': 'rmse',
#       'use_label_encoder': False
#       }
#       model = xgb.XGBRegressor(**params)

#     ml_forecast = MLForecast(models=[model],
#                    freq='H',
#                    lags=[1,2,12,24,168],
#                    date_features=['hour','day','week', 'month'],
#                    num_threads=6)

#     crossvalidation_df = ml_forecast.cross_validation(
#                                                       data=train,
#                                                       window_size=168,
#                                                       n_windows=3,
#                                                     )
#     rmse = crossvalidation_df.groupby(['unique_id', 'cutoff']).apply(lambda df: rmse(df['y'], df[model_name])).mean()

#     return rmse

# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=50, show_progress_bar=True)

# display(study.best_params)
# display(study.best_value)

Testing tree models

In [ ]:
xgb = XGBRegressor()
lgbm = lgb.LGBMRegressor()

ml_forecast = MLForecast(models=[xgb,lgbm],
                   freq='H',
                   lags=[1,2,3,12,24],
                   date_features=['hour','day','week'],
                   num_threads=6)

crossvalidation_df = ml_forecast.cross_validation(
    df=train_forecast,
    h=24,
    n_windows=3
)

for model_name in ml_forecast.models:
  rmse_value = crossvalidation_df.groupby(['unique_id', 'cutoff']).apply(lambda df: rmse(df['y'], df[model_name])).mean()

  display(model_name+" RMSE: "+str(rmse_value))

# ml_forecast.fit(train_forecast, static_features=['unique_id'], target_col='y')
# y_pred = ml_forecast.predict(h=len(test_forecast.index), X_df=test_forecast[exogenous_variables])

# for model_name in ml_forecast.models:
#   display(model_name+" RMSE: "+str(rmse(test_forecast['y'].reset_index(drop=True), y_pred[model_name])))

#   pd.Series(ml_forecast.models_[model_name].feature_importances_, index=ml_forecast.ts.features_order_).sort_values(ascending=False).plot.bar(figsize=(32, 6),title=f'{model_name} feature importance')
#   plt.show()

#   display(eli5.show_weights(ml_forecast.models_[model_name], feature_names = ml_forecast.ts.features_order_))

#   plt.figure(figsize=(30, 6))
#   sns.lineplot(x=test_forecast.index, y=test_forecast['y'], label='Real')
#   sns.lineplot(x=test_forecast.index, y=y_pred[model_name], label='Predicted')
#   plt.title('Real vs Predicted Values')
#   plt.legend()
#   plt.show()

DL model hyperparameter search

In [ ]:
# %%time
# warnings.filterwarnings('ignore')

# def objective(trial):

#     model_name = trial.suggest_categorical("classifier", ['LGBMRegressor', 'XGBRegressor'])

#     if model_name == "LGBMRegressor":
#       params = {
#         "objective": "regression",
#         "metric": "rmse",
#         "n_estimators": 1000,
#         "verbosity": -1,
#         "bagging_freq": 1,
#         "learning_rate": trial.suggest_float("learning_rate_light", 1e-3, 0.1, log=True),
#         "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
#         "subsample": trial.suggest_float("subsample_light", 0.05, 1.0),
#         "colsample_bytree": trial.suggest_float("colsample_bytree_light", 0.05, 1.0),
#         "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
#       }
#       model = lgb.LGBMRegressor(**params)

#     elif model_name == "XGBRegressor":
#       params = {
#       'max_depth': trial.suggest_int('max_depth', 1, 10),
#       'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
#       'n_estimators': trial.suggest_int('n_estimators', 50, 500),
#       'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
#       'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
#       'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
#       'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
#       'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
#       'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
#       'eval_metric': 'rmse',
#       'use_label_encoder': False
#       }
#       model = xgb.XGBRegressor(**params)

#     ml_forecast = MLForecast(models=[model],
#                    freq='H',
#                    lags=[1,2,12,24,168],
#                    date_features=['hour','day','week', 'month'],
#                    num_threads=6)

#     crossvalidation_df = ml_forecast.cross_validation(
#                                                       data=train,
#                                                       window_size=168,
#                                                       n_windows=3,
#                                                     )
#     rmse = crossvalidation_df.groupby(['unique_id', 'cutoff']).apply(lambda df: rmse(df['y'], df[model_name])).mean()

#     return rmse

# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=50, show_progress_bar=True)

# display(study.best_params)
# display(study.best_value)

DL model testing

In [ ]:
import os, torch, gc
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"
torch.cuda.empty_cache()
gc.collect()

In [ ]:
models = [LSTM(h=len(test_forecast.index),
               loss=DistributionLoss(distribution='Normal', level=[90]),
               max_steps=50,
               encoder_n_layers=2,
               encoder_hidden_size=200,
               context_size=8760,
               decoder_hidden_size=200,
               decoder_layers=2,
               learning_rate=1e-3,
               scaler_type='standard',
               hist_exog_list=train_forecast[train_forecast.columns.difference([target])],
               futr_exog_list=train_forecast[train_forecast.columns.difference([target])])]

neural_forecast = NeuralForecast(models=models, freq='H')

crossvalidation_df = neural_forecast.cross_validation(train_forecast, n_windows=3)

rmse = crossvalidation_df.groupby(['unique_id', 'cutoff']).apply(lambda df: rmse(df['y'], df['LSTM'])).mean()

display("RMSE: "+str(rmse))

#neural_forecast.fit(train_forecast)
# y_pred = neural_forecast.predict(futr_df=test_forecast[test_forecast.columns.difference(['y'])])

# for model_name in neural_forecast.models:
#   display(model_name+" RMSE: "+str(rmse(test_forecast['y'].reset_index(drop=True), y_pred[model_name])))

#   pd.Series(neural_forecast.models_[model_name].feature_importances_, index=neural_forecast.ts.features_order_).sort_values(ascending=False).plot.bar(figsize=(32, 6),title=f'{model_name} feature importance')
#   plt.show()

#   display(eli5.show_weights(neural_forecast.models_[model_name], feature_names = neural_forecast.ts.features_order_))

#   plt.figure(figsize=(30, 6))
#   sns.lineplot(x=test_forecast.index, y=test_forecast['y'], label='Real')
#   sns.lineplot(x=test_forecast.index, y=y_pred[model_name], label='Predicted')
#   plt.title('Real vs Predicted Values')
#   plt.legend()
#   plt.show()

In [ ]:
# y_pred =  neural_forecast.predict(futr_df=test)

# for model_name in neural_forecast.models:
#   display(mean_squared_error(test['y'], y_pred[model_name], squared=False))

#   pd.Series(neural_forecast.models_[model_name].feature_importances_, index=neural_forecast.ts.features_order_).sort_values(ascending=False).plot.bar(figsize=(32, 6),title=f'{model_name} feature importance')
#   plt.show()

#   plt.figure(figsize=(30, 6))
#   sns.lineplot(x=test.index, y=test['y'], label='Real')
#   sns.lineplot(x=test.index, y=y_pred[model_name], label='Predicted')
#   plt.title('Real vs Predicted Values')
#   plt.legend()
#   plt.show()